TRADE IN GOODS STATISTICS:Employee Count and age

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
import requests
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.uktradeinfo.com/statistics/documents/idbr_ots_tables_2015.xls'
inputFile = sourceFolder / 'idbr_ots_tables_2015.xls'
if not(inputFile.exists() and inputFile.is_file()):
    response = requests.get(inputURL)
    with open(inputFile, 'wb') as f:
        f.write(response.content)

In [3]:
tab = loadxlstabs(inputFile)

Loading in\idbr_ots_tables_2015.xls which has size 174080 bytes
Table names: ['Notes and Contents', 'Industry Group', 'Age Group', 'Employee Size', 'Industry_Age', 'Industry_EmployeeSize', 'EmployeeSize_Age', 'Metadata']


In [4]:
tab = tab[6]

In [5]:
savepreviewhtml(tab)

TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,
Release Date 24/11/2016,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,
"Total value of UK Trade, Business count and Employee Count, by Employee Size group and Age for Imports and Exports",,,,,,,,,,,,,,,,,,,,,
,,,Exports,,,,Imports,,,,,,,,,,,,,,
,,,2015.0,,,,2015.0,,,,,,,,,,,,,,
Employees,Age (years),,Value £ m,Business Count,Employee Count,,Value £ m,Business Count,Employee Count,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,
0.0,0 to 1,,1052.0,1083.0,0.0,,2348.0,2615.0,0.0,,,,,,,,,,,,
,2 to 3,,1909.0,1123.0,0.0,,3828.0,2036.0,0.0,,,,,,,,,,,,


In [6]:
observations = tab.excel_ref('D10').expand(DOWN).expand(RIGHT).is_not_blank() 

In [7]:
observations = observations - tab.excel_ref('E10').expand(DOWN).is_not_blank()

In [8]:
observations = observations - tab.excel_ref('F10').expand(DOWN).is_not_blank()

In [9]:
observations = observations - tab.excel_ref('J10').expand(DOWN).is_not_blank()

In [10]:
observations = observations - tab.excel_ref('I10').expand(DOWN).is_not_blank()

In [11]:
totalemployees = tab.excel_ref('A10').expand(DOWN).is_not_blank()

In [12]:
age = tab.excel_ref('B10').expand(DOWN)

In [13]:
Flow = tab.excel_ref('D6').expand(RIGHT).is_not_blank()

In [14]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDimConst('Year','2015'),
            HDimConst('Unit', '£ Millions'), 
            HDimConst('Measure Type','GBP Total'),            
            HDim(totalemployees, 'Total Employees', CLOSEST, ABOVE),
            HDim(Flow, 'Flow', CLOSEST, LEFT),
            HDim(age, 'Employees Age(Yrs)', DIRECTLY, LEFT)
]

In [15]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [16]:
savepreviewhtml(c1)

In [17]:
new_table = c1.topandas()

In [18]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)

In [19]:
new_table.head(5)

,Value,DATAMARKER,Geography,Year,Unit,Measure Type,Total Employees,Flow,Employees Age(Yrs)
0,1052,NaN,K02000001,2015,£ Millions,GBP Total,0.0,Exports,0 to 1
1,2348,NaN,K02000001,2015,£ Millions,GBP Total,0.0,Imports,0 to 1
2,1909,NaN,K02000001,2015,£ Millions,GBP Total,0.0,Exports,2 to 3
3,3828,NaN,K02000001,2015,£ Millions,GBP Total,0.0,Imports,2 to 3
4,,S,K02000001,2015,£ Millions,GBP Total,0.0,Exports,4 to 5


In [21]:
new_table['Total Employees'] = new_table['Total Employees'].str.strip()
new_table['Year'] = new_table['Year'].str.strip()
new_table['Flow'] = new_table['Flow'].str.strip()
new_table['Unit'] = new_table['Unit'].str.strip()
new_table['Measure Type'] = new_table['Measure Type'].str.strip()
new_table['Geography'] = new_table['Geography'].str.strip()
new_table['Employees Age(Yrs)'] = new_table['Employees Age(Yrs)'].str.strip()

In [22]:
new_table = new_table[['Geography','Year','Total Employees','Flow','Employees Age(Yrs)','Measure Type','Value','Unit']]

In [23]:
new_table['Total Employees'] = new_table['Total Employees'].map(lambda cell:cell.replace('Grand Total9', 'Grand Total'))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
new_table = new_table[new_table['Value'] != '']

In [25]:
new_table = new_table[new_table['Value'] != 'S']

In [26]:
def user_perc(x):
    
    if x.strip(' ') == '':
        return 'NA'
    else:
        return x
    
new_table['Employees Age(Yrs)'] = new_table.apply(lambda row: user_perc(row['Employees Age(Yrs)']), axis = 1)

In [27]:
new_table.head(4)

,Geography,Year,Total Employees,Flow,Employees Age(Yrs),Measure Type,Value,Unit
0,K02000001,2015,0.0,Exports,0 to 1,GBP Total,1052,£ Millions
1,K02000001,2015,0.0,Imports,0 to 1,GBP Total,2348,£ Millions
2,K02000001,2015,0.0,Exports,2 to 3,GBP Total,1909,£ Millions
3,K02000001,2015,0.0,Imports,2 to 3,GBP Total,3828,£ Millions


In [28]:
new_table.tail(4)

,Geography,Year,Total Employees,Flow,Employees Age(Yrs),Measure Type,Value,Unit
70,K02000001,2015,Unknown,Exports,Unknown,GBP Total,21915,£ Millions
71,K02000001,2015,Unknown,Imports,Unknown,GBP Total,12763,£ Millions
72,K02000001,2015,Grand Total,Exports,NA,GBP Total,304768,£ Millions
73,K02000001,2015,Grand Total,Imports,NA,GBP Total,411033,£ Millions


In [29]:
out = Path('out')
out.mkdir(exist_ok=True)
new_table.to_csv(out / 'employeecountagevalue.csv', index = False)